Lets import needed libraries for our project.

In [5]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from IPython.display import Markdown, display

Lets define our constants. Ollama local host, headers and model.

In [6]:
OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

Lets create a "Website" class, which will webscrap the desired site. To do that we will use BeautifulSoup library.

In [7]:
class Website:
    
    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

Lets define our task for LLM as a promt.

In [8]:
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

Here we will define a function that writes a User Prompt that asks for summaries of websites.

In [9]:
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

This function defines the format

In [10]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

This function actually talks with Ollama model.

In [11]:
def summarize(url):
    website = Website(url)
    payload = {
        "model": MODEL,
        "messages": messages_for(website),
        "stream": False
    }
    
    response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
    return response.json()['message']['content']

Lets call the function "summarize".

In [12]:
summarize("https://stytch.com/")

"### Summary of Stytch\n\n**Mission**\nBuild auth that's seamless, easy to implement, and scalable for any scale.\n\n**Product Features**\n\n*   **Developer Infrastructure**: Pre-built UI, headless frontend SDKs, backend APIs and SDKs, purpose-built options for consumer and multi-tenant apps.\n*   **Fraud and Risk Prevention**: Fingerprinting with 99.99% accurate bot detection, reliable risk verdicts to Allow, Block or Challenge visitors, strong invisible CAPTCHA, adaptive MFA.\n*   **Future-proof Infrastructure**: Scalable user + org data models, secure account deduplication, primary and secondary auth options with email and SMS provider failover.\n\n**Benefits**\n\n*   Simplify auth for developers\n*   Reduce time spent on auth (tenth of original time)\n*   Support both SAML and OIDC protocols for Single Sign On\n\n### News and Announcements\n\n*   Tens of millions of users migrated from Auth0 to Stytch in under a month.\n\n**Comparison**\n\nStytch vs. other auth providers:\n*   [Sty

Here we will create a function to display the result nicely in the Jupyter output, using markdown.

In [13]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

Lets call "display_summary" function.

In [14]:
display_summary("https://stytch.com/")

# Stytch - A Better Way to Build Authentication

## Overview

Stytch is a cloud-based authentication platform designed to simplify the process of building and managing authentication flows for web, mobile, and backend applications. The platform offers a range of features, including pre-built UIs, APIs, SDKs, and fraud prevention capabilities.

### Key Features

* Pre-built UIs and SDKs for authentication and authorization
* Flexible developer infrastructure with unparalleled flexibility
* Fraud and risk prevention capabilities with 99.99% accurate bot detection
* Scalable user and organization data models
* Support for Single Sign-On (SSO) protocols like SAML and OIDC
* Multi-factor authentication options, including RBAC, JIT Provisioning, and more

## News and Announcements

### Recent Migration Success Stories

Stytch has successfully migrated tens of millions of users from Auth0 to Stytch in under a month. This migration showcases the platform's ease of use and scalability.

### Developer Testimonials

* "We cut the time spent on authentication flows by 75% after switching to Stytch." - [Developer Testimonial]
* "Stytch's pre-built UIs saved us weeks of development time. We love it!" - [Developer Testimonial]

## Industries Served

Stytch serves a variety of industries, including:

* SaaS
* Ecommerce
* Fintech
* Web3
* Consumer tech
* Healthcare